In [1]:
import os
import csv
import json
import requests
import pandas as pd

In [4]:
data_path = "/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset"
train_csv_path = os.path.join(data_path, "train_split_Depression_AVEC2017.csv")
dev_csv_path = os.path.join(data_path, "dev_split_Depression_AVEC2017.csv")
df_train = pd.read_csv(train_csv_path)
df_dev = pd.read_csv(dev_csv_path)
print(df_train.head(5))
id_list = list(df_train["Participant_ID"])
print(id_list)

count_control = sum(df_train['PHQ8_Score']<=4) + sum(df_dev['PHQ8_Score']<=4)
count_mild = sum((df_train['PHQ8_Score']>=5) & (df_train['PHQ8_Score']<=9)) + sum((df_dev['PHQ8_Score']>=5) & (df_dev['PHQ8_Score']<=9))
count_moderate = sum((df_train['PHQ8_Score']>=10) & (df_train['PHQ8_Score']<=14)) + sum((df_dev['PHQ8_Score']>=10) & (df_dev['PHQ8_Score']<=14))
count_moderately_severe = sum((df_train['PHQ8_Score']>=15) & (df_train['PHQ8_Score']<=19)) + sum((df_dev['PHQ8_Score']>=15) & (df_dev['PHQ8_Score']<=19))
count_severe = sum(df_train['PHQ8_Score']>=20) + sum(df_dev['PHQ8_Score']>=20)
print(f"Control: {count_control}, Mild: {count_mild}, Moderate: {count_moderate}, Moderately Severe: {count_moderately_severe}, Severe: {count_severe}")

   Participant_ID  PHQ8_Binary  PHQ8_Score  Gender  PHQ8_NoInterest  \
0             303            0           0       0                0   
1             304            0           6       0                0   
2             305            0           7       1                0   
3             310            0           4       1                1   
4             312            0           2       1                0   

   PHQ8_Depressed  PHQ8_Sleep  PHQ8_Tired  PHQ8_Appetite  PHQ8_Failure  \
0               0         0.0           0              0             0   
1               1         1.0           2              2             0   
2               1         1.0           2              2             1   
3               1         0.0           0              0             1   
4               0         1.0           1              0             0   

   PHQ8_Concentrating  PHQ8_Moving  
0                   0            0  
1                   0            0  
2                

In [ ]:
def load_transcript(data_path, id):
    transcript_path = os.path.join(data_path, f"{id}_P/{id}_TRANSCRIPT.csv")
    df = pd.read_csv(transcript_path)
    with open(transcript_path, 'r') as file:
        header = file.readline().split("\t")
        interview = ""
        for line in file:
            parts = line.split("\t")
            try:
                interview += parts[2]+": "+parts[3]
            except IndexError:
                print(line)
    return interview

In [ ]:
id = 315
interview = load_transcript(data_path, id)
print(interview)

Ellie: hi i'm ellie thanks for coming in today i was created to talk to people in a safe and secure environment think of me as a friend i don't judge i can't i'm a computer
Ellie: i''m here to learn about people and would love to learn about you i'll ask a few questions to get us started and please feel free to tell me anything your answers are totally confidential
Ellie: how are you doing today
Participant: i'm doing good um
Ellie: that's good
Ellie: where are you from originally
Participant: from los angeles california
Ellie: nice
Ellie: what are some things you really like about l_a
Participant: um the cool weather the beaches the people the schools 
Ellie: mhm
Ellie: what'd you study at school 
Participant: um child development
Ellie: nice are you still doing that
Participant: yes
Ellie: what's your dream job
Participant: to be a teacher for special education
Ellie: how hard is that
Participant: it's not hard 'cause i love to work with children
Ellie: really
Participant: yes
Ellie:

In [ ]:
def chat_completion(messages, model="Llama-4-Maverick-17B-128E-Instruct-FP8", max_tokens=256):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {LLAMA_API_KEY}"
    }

    payload = {
        "messages": messages,
        "model": model,
        "max_tokens": max_tokens,
        "stream": False
    }
    
    response = requests.post(
        f"{BASE_URL}/chat/completions", 
        headers=headers, 
        json=payload
    )

    return response.json()

LLAMA_API_KEY = "LLM|1251083943113479|Pj9PRcO4pbzdq6BRMR3dqCN3tq8"
BASE_URL = "https://api.llama.com/v1" # llama4 server URL
# BASE_URL = "arctrdagn033.rs.gsu.edu:11434" # ollama server URL

system_prompt = "You are an AI psychiatrist assistant specializing in depression and anxiety. Your task is to analyze interview transcripts and assess the patient's mental health status, focusing on signs of depression."

qualitative_assessment_prompt = f"""Here is the interview transcript:
{interview}

Please analyze the interview transcript above and assess the patient's mental health status, focusing on signs of depression, from social and biological aspects. Please think step-by-step and provide your assessment in a structured format, including the following sections:
1. Overall Impression in <overall> tags: A brief summary of the patient's mental health status.
2. Social Aspects in <social> tags: Observations related to the patient's interpersonal relationships, family situation, and any relevant social interactions that may influence their mental health. Find relevant quotes from the interview transcript.
3. Biological Aspects in <biological> tags: Observations related to the patient's physical health, including sleep quality, appetite, physical activity, stress level, and any other biological factors that may influence their mental health. Find relevant quotes from the interview transcript.
4. Additional Notes in <notes> tags: Any other relevant observations or comments that do not fit into the above categories.
5. Potential Risk Factors in <risks> tags: Any identified risk factors for depression or other mental health issues."""

qualitative_assessment_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": qualitative_assessment_prompt}
]

qualitative_assessment_response = chat_completion(qualitative_assessment_messages)
qualitative_assessment_response_text = qualitative_assessment_response["completion_message"]["content"]["text"]
print(qualitative_assessment_response_text)

## Step 1: Analyze the Overall Impression
The participant appears to be generally positive and open during the interview, discussing various aspects of their life, including their interests, relationships, and future plans. They express happiness and contentment with their current situation, although there are some areas of concern.

## Step 2: Examine Social Aspects
The participant discusses their close relationship with their family, particularly their mother, stating, "my mom... we're very close we talk every day." They also mention being in a relationship with a boyfriend they've known for six months, indicating a level of social connection. However, they express some regret over their current living situation, having chosen roommates who are "very quiet" and not as connected to as they had hoped. The participant also mentions a recent argument with their sister, feeling "very betrayed and belittled" due to a misunderstanding and the sister's actions on social media.

## Step 3: In

In [ ]:
quantitative_assessment_prompt = f"""Here is the interview transcript:
{interview}

Please analyze the interview transcript above, assess the patient's signs of depression, and predict the eight-item Patient Health Questionnaire depression scale (PHQ-8) scores. The PHQ-8 counts the number of days in the past 2 weeks the respondent had experienced a particular depressive symptom, and assigns points (0 to 3) to each category: (1) 0 point: 0 to 1 day ("not at all"); (2) 1 point: 2 to 6 days ("several days"); (3) 2 points: 7 to 11 days ("more than half the days"); (4) 3 points: 12 to 14 days ("nearly every day"). Please think step-by-step and provide your assessment in a structured format, including the following assessments:
1. When the patient reported little interest or pleasure in doing things, find relevant quotes in <nointerest_quotes> tags, assess relevant symptoms in <nointerest_assessment> tags, and only provide an integer score in <nointerest_score> tags.
2. When the patient reported feeling down, depressed, or hopeless, find relevant quotes in <depressed_quotes> tags, assess relevant symptoms in <depressed_assessment> tags, and only provide an integer score in <depressed_score> tags.
3. When the patient reported trouble falling or staying asleep, or sleeping too much, find relevant quotes in <sleep_quotes> tags, assess relevant symptoms in <sleep_assessment> tags, and only provide an integer score in <sleep_score> tags.
4. When the patient reported feeling tired or having little energy, find relevant quotes in <tired_quotes> tags, assess relevant symptoms in <tired_assessment> tags, and only provide an integer score in <tired_score> tags.
5. When the patient reported poor appetite or overeating, find relevant quotes in <appetite_quotes> tags, assess relevant symptoms in <appetite_assessment> tags, and only provide an integer score in <appetite_score> tags.
6. When the patient reported feeling bad about themselves, or that they are a failure or have let themselves or their family down, find relevant quotes in <failure_quotes> tags, assess relevant symptoms in <failure_assessment> tags, and only provide an integer score in <failure_score> tags.
7. When the patient reported trouble concentrating on things, such as reading the newspaper or watching television, find relevant quotes in <concentrating_quotes> tags, assess relevant symptoms in <concentrating_assessment> tags, and only provide an integer score in <concentrating_score> tags.
8. When the patient reported moving or speaking so slowly that other people could have noticed, or the opposite - being so fidgety or restless that they have been moving around a lot more than usual, find relevant quotes in <moving_quotes> tags, assess relevant symptoms in <moving_assessment> tags, and only provide an integer score in <moving_score> tags."""

quantitative_assessment_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quantitative_assessment_prompt}
]

quantitative_assessment_response = chat_completion(quantitative_assessment_messages)
quantitative_assessment_response_text = quantitative_assessment_response["completion_message"]["content"]["text"]
print(quantitative_assessment_response_text)

To assess the patient's mental health status and predict the PHQ-8 scores based on the interview transcript, we will analyze the conversation step by step, focusing on signs of depression.

### 1. Little interest or pleasure in doing things

<nointerest_quotes>
- "it's not hard 'cause i love to work with children" - This suggests she enjoys her work, indicating pleasure in doing things.
- "being with my boyfriend makes me really happy" - Indicates she derives happiness from certain activities.
</nointerest_quotes>

<nointerest_assessment>
The participant expresses enjoyment in her work with children and happiness when with her boyfriend, suggesting she has interest and pleasure in doing certain things. There's no clear indication of a lack of interest or pleasure.
</nointerest_assessment>

<nointerest_score>0</nointerest_score>

### 2. Feeling down, depressed, or hopeless

<depressed_quotes>
- "i felt very betrayed and belittled" - This indicates a negative emotional state but is relat

In [20]:
def extract_response(response_text, key_list):
    output_dict = {}
    for k in key_list:
        idx1 = response_text.index(f"<{k}>")
        idx2 = response_text.index(f"</{k}>")
        res = response_text[idx1 + len(f"<{k}>"): idx2]
        print(res)
        if "score" in k:
            output_dict[k] = int(res)
        else:
            output_dict[k] = res
    return output_dict

qualitative_assessment_key_list = ["overall", "social", "biological", "notes", "risks"]
qualitative_assessment_dict = extract_response(qualitative_assessment_response_text, qualitative_assessment_key_list)

quantitative_assessment_key_list = []
for i in ["nointerest", "depressed", "sleep", "tired", "appetite", "failure", "concentrating", "moving"]:
    for j in ["quotes", "assessment", "score"]:
        quantitative_assessment_key_list.append(f"{i}_{j}")
quantitative_assessment_dict = extract_response(quantitative_assessment_response_text, quantitative_assessment_key_list)

The participant appears to be generally mentally healthy with some areas of stress and potential for emotional vulnerability.
The participant has a close relationship with their family, especially their mother, and is in a romantic relationship. They have some social connections and friendships. However, they expressed regret over their current living situation and had a recent argument with their sister, indicating some social stressors. Relevant quotes include: "my mom... we're very close we talk every day" and "i felt very betrayed and belittled" during the argument with their sister.
The participant reports some difficulty with sleep, stating it's "pretty good... eh somewhat" when asked about getting a good night's sleep. They admit to being tired and falling asleep during class when they don't sleep well. They attribute their lack of sleep to staying up late.
The participant does not report a diagnosis of depression or PTSD. They are optimistic about traveling and have plans for t

In [17]:
output_dict = {'participant_id': id, 'qualitative_assessment': qualitative_assessment_response_text, 'quantitative_assessment': quantitative_assessment_response_text}
output_dict.update(qualitative_assessment_dict)
output_dict.update(quantitative_assessment_dict)

output_dict

{'participant_id': 304,
 'qualitative_assessment': '## Step 1: Analyze the Overall Impression\nThe participant appears to be generally positive and open during the interview, discussing various aspects of their life, including their interests, relationships, and future plans. They express happiness and contentment with their current situation, although there are some areas of concern.\n\n## Step 2: Examine Social Aspects\nThe participant discusses their close relationship with their family, particularly their mother, stating, "my mom... we\'re very close we talk every day." They also mention being in a relationship with a boyfriend they\'ve known for six months, indicating a level of social connection. However, they express some regret over their current living situation, having chosen roommates who are "very quiet" and not as connected to as they had hoped. The participant also mentions a recent argument with their sister, feeling "very betrayed and belittled" due to a misunderstandin

In [ ]:
data = [
    output_dict
]

with open('assessment.csv', 'w', newline='') as csvfile:
    fieldnames = ['participant_id', 'qualitative_assessment', 'quantitative_assessment']
    fieldnames += qualitative_assessment_key_list
    fieldnames += quantitative_assessment_key_list
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)